In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import copy
import time
from torch.cuda.amp import GradScaler
from EarlyStopping import EarlyStopping
from Common_Function_ import *
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

GPU = '0,1,2,3'
os.environ["CUDA_VISIBLE_DEVICES"] = GPU
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
# set_seeds()

In [2]:
calculate = False
EPOCHS = 50
BATCH_SIZE = 64
VALID_RATIO = 0.3
N_IMAGES = 100
START_LR = 1e-5
END_LR = 10
NUM_ITER = 100
PATIENCE_EARLYSTOP=10

pretrained_size = 224
pretrained_means = [0.4489, 0.3352, 0.3106]#[0.485, 0.456, 0.406]
pretrained_stds= [0.2380, 0.1965, 0.1962]#[0.229, 0.224, 0.225]

In [3]:
class CustumDataset(Dataset):
    def __init__(self, data, target, data_2=None, target_2=None,transform=None):
        self.data = data
        self.target = target
        self.data_video = data_2    
        self.target_video = target_2
        self.transform = transform
        
        if self.data_video:
            self.len_data2 = len(self.data_video)
        print(self.len_data2)
        print(len(self.data_video))
        print(len(self.data))
        
        assert(self.len_data2 == len(self.target) == len(self.target_video) == len(self.data) == len(self.data_video))
    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        path = self.data[idx]
        img = Image.open(path)
        img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
            
        if self.data_video:
            path_video = self.data[idx]
            img_video = Image.open(path_video)
            img_video = img_video.convert('RGB')
            if self.transform:
                img_video = self.transform(img_video)
            return img, self.target[idx], img_video, self.target_video[idx]
        return img, self.target[idx]

In [4]:
train_transforms = transforms.Compose([
                           transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.RandomHorizontalFlip(0.5),
                           # transforms.RandomCrop(pretrained_size, padding = 10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means,
                                                std = pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])

train_dir = ["/media/data1/mhkim/FAKEVV_hasam/train/SPECTOGRAMS/real_A_fake_B",
             "/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC"]
test_dir =  ["/media/data1/mhkim/FAKEVV_hasam/test/SPECTOGRAMS/real_A_fake_B",
             "/media/data1/mhkim/FAKEVV_hasam/test/FRAMES/real_A_fake_MergedC"]

list_train = [datasets.ImageFolder(root = train_dir[0],transform = None),
             datasets.ImageFolder(root = train_dir[1],transform = None)]
list_test = [datasets.ImageFolder(root = test_dir[0],transform = None),
            datasets.ImageFolder(root = test_dir[1],transform = None)]


list_glob_trainpath_video = [list_train[0].samples[i][0] for i in range(len(list_train[0].samples))]
list_glob_trainpath = [list_train[1].samples[i][0] for i in range(len(list_train[1].samples))]
list_targets_trainpath_video = [list_train[0].targets[i] for i in range(len(list_train[0].targets))]
list_targets_trainpath = [list_train[1].targets[i] for i in range(len(list_train[1].targets))]

list_glob_testpath_video = [list_test[0].samples[i][0] for i in range(len(list_test[0].samples))]
list_glob_testpath = [list_test[1].samples[i][0] for i in range(len(list_test[1].samples))]
list_targets_testpath_video = [list_test[0].targets[i] for i in range(len(list_test[0].targets))]
list_targets_testpath = [list_test[1].targets[i] for i in range(len(list_test[1].targets))]
# print(list_targets_testpath[3402])
# print(list_targets_testpath_video.count(0))
# print(len(list_targets_testpath_video))

In [5]:
###################TARGET##################
cnt_target_video = list_targets_testpath_video.count(0)
cnt_target = list_targets_testpath.count(0)
temp_list = list_targets_testpath_video[:cnt_target_video]
temp_list_input = list_glob_testpath_video[:cnt_target_video]
head = cnt_target//cnt_target_video
print('head is',head)
list_0 , list_1 = {'input':[],'target':[]},{'input':[],'target':[]}
for c in range(head):
    list_0['target']+=temp_list
    list_0['input']+=temp_list_input
list_0['target']+=temp_list[:(cnt_target-len(list_0['target']))]
list_1['target']+=list_targets_testpath[cnt_target:]
list_0['input']+=temp_list_input[:(cnt_target-len(list_0['input']))]
list_1['input']+=list_glob_testpath[cnt_target:]


list_test_target_videos = list_0['target'] + list_1['target']
list_test_input_videos = list_0['input'] + list_1['input']
# print(len(list_test_target_videos))
print(list_test_target_videos==list_targets_testpath)

assert((len(list_test_target_videos))==(len(list_targets_testpath))==len(list_test_input_videos))

print(len(list_test_target_videos))
print(len(list_test_input_videos))
print(len(list_targets_testpath))

##################TRAIN####################
cnt_train_video = list_targets_trainpath_video.count(0)
cnt_train = list_targets_trainpath.count(0)
temp_list = list_targets_trainpath_video[:cnt_train_video]
temp_list_input = list_glob_trainpath_video[:cnt_train_video]
head = cnt_train//cnt_train_video
print('head is',head)
list_0 , list_1 = {'input':[],'target':[]},{'input':[],'target':[]}
for c in range(head):
    list_0['target']+=temp_list
    list_0['input']+=temp_list_input
list_0['target']+=temp_list[:(cnt_train-len(list_0['target']))]
list_1['target']+=list_targets_trainpath[cnt_train:]
list_0['input']+=temp_list_input[:(cnt_train-len(list_0['input']))]
list_1['input']+=list_glob_trainpath[cnt_train:]

list_train_target_videos = list_0['target'] + list_1['target']
list_train_input_videos = list_0['input'] + list_1['input']
print(list_train_target_videos==list_targets_trainpath)
assert((len(list_train_target_videos))==(len(list_targets_trainpath))==len(list_train_input_videos))

print(len(list_train_target_videos))
print(len(list_targets_trainpath))
print(len(list_train_input_videos))


head is 34
True
6806
6806
6806
head is 176
True
127356
127356
127356


In [6]:
#image, video 한꺼번에 저장
# list_glob_trainpath = [list_train[0].samples[i][0] for i in range(len(list_train[0].samples))]
# list_glob_trainpath += [list_train[1].samples[i][0] for i in range(len(list_train[1].samples))]
# list_targets_trainpath = [list_train[0].targets[i] for i in range(len(list_train[0].targets))]
# list_targets_trainpath += [list_train[1].targets[i] for i in range(len(list_train[1].targets))]
# list_glob_testpath = [list_test[0].samples[i][0] for i in range(len(list_test[0].samples))]
# list_glob_testpath += [list_test[1].samples[i][0] for i in range(len(list_test[1].samples))]
# list_targets_testpath = [list_test[0].targets[i] for i in range(len(list_test[0].targets))]
# list_targets_testpath += [list_test[1].targets[i] for i in range(len(list_test[1].targets))]
###########################
# train_data_video = CustumDataset(list_glob_trainpath_video, list_targets_trainpath_video, test_transforms)

# n_valid_examples_video = int(len(train_data_video) * 0.2)
# n_train_examples_video = len(train_data_video) - n_valid_examples_video
# test_data_video = CustumDataset(list_glob_testpath_video, list_targets_testpath_video, test_transforms)

train_data = CustumDataset(list_glob_trainpath, list_targets_trainpath, list_train_input_videos, list_train_target_videos,test_transforms)
n_valid_examples = int(len(train_data) * VALID_RATIO)#기존 test data자체가 너무 적어서 train기준으로 비율조정
n_train_examples = len(train_data) - n_valid_examples

train_data, valid_data = data.random_split(train_data, [n_train_examples, n_valid_examples])
# train_data_video, valid_data_video = data.random_split(train_data_video, [n_train_examples_video, n_valid_examples_video])

train_data.transform = train_transforms; train_data.dataset.transform = train_transforms
valid_data.transform = test_transforms; valid_data.dataset.transform = test_transforms

test_data = CustumDataset(list_glob_testpath, list_targets_testpath, list_test_input_videos, list_test_target_videos, test_transforms)
print(len(list_glob_testpath_video), len(list_targets_testpath_video))
print(len(list_test_input_videos), len(list_test_target_videos))
from PIL import Image
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_data, 
                                shuffle = True, 
                                batch_size = BATCH_SIZE)

127356
127356
127356
6806
6806
6806
198 198
6806 6806
Number of training examples: 89150
Number of validation examples: 38206
Number of testing examples: 6806


In [7]:
class FeatureDataset(Dataset):
    def __init__(self,featlst1, featlst2, labellst):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst1[index], self.featlst2[index], self.labellst[index])
    
    def __len__(self):
        return len(self.labellst)


def Get_loader_feats(models, train_iterator, valid_iterator, test_iterator,device):
    trn_labels = []
    trn_features = []
    val_labels = []
    val_features = []
    test_labels = []
    test_features = []
    
    for m in models:
        _trn_la, _trn_fea, _val_la, _val_fea, _test_la, _test_fea = [],[],[],[],[],[]
        m.to(device)
        m.train()
        for d, la in tqdm(train_iterator):
            o,_ = m(Variable(d.to(device)))
            o = o.view(o.size(0),-1)
            _trn_la.append(la)
            _trn_fea.append(o)
        
        for d,la in tqdm(valid_iterator):
            with torch.no_grad():
                o,_= m(Variable(d.to(device)))
                o= o.view(o.size(0),-1)
                _val_la.append(la)
                _val_fea.append(o)
        
        for d,la in tqdm(test_iterator):
            with torch.no_grad():
                o,_= m(Variable(d.to(device)))
                o= o.view(o.size(0),-1)
                _test_la.append(la)
                _test_fea.append(o)
            
        trn_labels.append(_trn_la)
        trn_features.append(_trn_fea)
        val_labels.append(_val_la)
        val_features.append(_val_fea)
        test_labels.append(_test_la)
        test_features.append(_test_fea)
        
    trn_feat_dset =FeatureDataset(trn_features[0], trn_features[1], trn_labels)
    val_feat_dset =FeatureDataset(val_features[0], val_features[1], val_labels)
    test_feat_dset =FeatureDataset(test_features[0], test_features[1], test_labels)
    
    trn_feat_loader = DataLoader(trn_feat_dset, batch_size=BATCH_SIZE, shuffle=True)
    val_feat_loader = DataLoader(val_feat_dset, batch_size=BATCH_SIZE)
    test_feat_loader = DataLoader(test_feat_dset, batch_size=BATCH_SIZE)

    print("model loader done")
    
    return trn_feat_loader, val_feat_loader, test_feat_loader

In [19]:
from torchvision.models import resnet50
model = resnet50()
print(f'number of train/val/test loader : {len(train_iterator), len(valid_iterator), len(test_iterator)}')
models = [resnet50(), resnet50()]
MODELS_NAME = 'ResNet50'
#checkpoinsts for model loaders :  [VIDEO(A&B), FRAME(A&C)]
list_checkpoint = [torch.load(f'/home/mhkim/DFVV/{MODELS_NAME}_realA_fakeB.pt')['state_dict'],
                   torch.load(f'/home/mhkim/DFVV/{MODELS_NAME}_realA_fakeC.pt')['state_dict']]
models[0].load_state_dict(list_checkpoint[0])
models[1].load_state_dict(list_checkpoint[1])
OUTPUT_DIM = 2
criterion = nn.CrossEntropyLoss().to(device)
scaler = GradScaler()
early_stopping = EarlyStopping(patience=PATIENCE_EARLYSTOP, verbose=True)


number of train/val/test loader : (1393, 597, 107)


In [43]:
from torchsummary import summary
test = resnet50()
summary(resnet50().cuda(),(3,224,224))
modules=list(test.children())[:-1]
test=nn.Sequential(*modules)
summary(test.cuda(),(3,224,224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [35]:
class Ensemble(nn.Module):
    def __init__(self,models,device,training=True):
        super().__init__()
        self.model1 = models[0] #FOR VIDEO
        self.model2 = models[1] #FOR FRAME IMAGE
        self.fc1 = nn.Linear(2048 ,224)#.to(device)
        self.fc2 = nn.Linear(2048, 224)#.to(device)
        self.fc = nn.Linear(224,2).to(device)
#         selr.drop 
    def forward(self, frame, video):
        feat1 = self.model1(video)
        feat2 = self.model2(frame)
        out = (feat1 + feat2)/2
#         inp1 = feat1.view(feat1.size(0), -1)
#         inp2 = feat2.view(feat2.size(0), -1)
#         out1 = self.fc1(inp1)
#         out2 = self.fc2(inp2)
        out = out1 + out2
        out = out/2
        out = self.fc(out)
        return out
    
ecls = Ensemble(models,device)
ecls = nn.DataParallel(ecls)   # 4개의 GPU를 이용할 경우
ecls = ecls.to(device)

In [39]:
optimizer = optim.Adam(ecls.parameters(), lr = START_LR)
scaler = GradScaler()
STEPS_PER_EPOCH = len(train_iterator)
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH
best_valid_loss = float('inf')

In [37]:
import copy
def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    output=[]
    model.train()
    correct, total = 0,0
    
    for (x, y, x_v, y_v) in tqdm(iterator):
        with autocast(enabled=True):
            x, y, x_v, y_v = x.to(device), y.to(device), x_v.to(device), y_v.to(device)
            optimizer.zero_grad()
            final_output = model(x, x_v)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = torch.max(final_output.data, 1)
            correct += predicted.eq(y.data).cpu().sum()
            total += y.size(0)
        
            epoch_loss += loss.item()
    print("Accuracy = {}".format(100. * correct / total))

    epoch_loss /= len(iterator)
    epoch_acc = correct / total 
    return epoch_loss, epoch_acc


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for (x, y, x_v, y_v) in tqdm(iterator):
            x, y, x_v, y_v = x.to(device), y.to(device), x_v.to(device), y_v.to(device)
            final_output = model(x, x_v)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            _, predicted = torch.max(final_output.data, 1)
            correct += predicted.eq(y.data).cpu().sum()
            total += y.size(0)
            epoch_loss += loss.item()
    epoch_loss /= len(iterator)
    epoch_acc = correct / total

    return epoch_loss, epoch_acc

In [38]:
print("training...")
for epoch in range(EPOCHS):
    start_time = time.monotonic()
    train_loss, train_acc = train_voting(ecls, train_iterator, optimizer, criterion, scaler, device)
    valid_loss, valid_acc = evaluate_voting(ecls, valid_iterator, criterion, device)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save({'state_dict':ecls.state_dict(),
                   'best_acc':valid_acc,
                   'val_loss':valid_loss,
                   'epoch':epoch,
                   'lr':START_LR,
                   'best_acc':valid_acc,
                   }, f'[SFVoting,{MODELS_NAME}]with_pretrained.pt')

    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc @1: {train_acc*100:6.2f}% | ')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc @1: {valid_acc*100:6.2f}% | ')
    
    if early_stopping:
        early_stopping(valid_loss, ecls)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    

training...


  0%|                      | 0/1393 [00:00<?, ?it/s]

torch.Size([16, 1000])torch.Size([16, 1000])

torch.Size([16, 1000])
torch.Size([16, 1000])


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/mhkim/anaconda3/envs/mhmh/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/mhkim/anaconda3/envs/mhmh/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/tmp/ipykernel_35275/547018674.py", line 16, in forward
    out1 = self.fc1(inp1)
  File "/home/mhkim/anaconda3/envs/mhmh/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/mhkim/anaconda3/envs/mhmh/lib/python3.8/site-packages/torch/nn/modules/linear.py", line 94, in forward
    return F.linear(input, self.weight, self.bias)
  File "/home/mhkim/anaconda3/envs/mhmh/lib/python3.8/site-packages/torch/nn/functional.py", line 1753, in linear
    return torch._C._nn.linear

trn_feat_loader, val_feat_loader, test_feat_loader = Get_loader_feats(models,train_iterator,valid_iterator,test_iterator,device)

from sklearn import model_selection
from PIL import Image
from tqdm import tqdm
labels = ['model1', 'model2']
device= 'cuda'
ecls.train()
for i, (x,y) in enumerate(tqdm(train_iterator)):
    print(y)
    
    with autocast(enabled=True):
        x = x.to(device); y= y.to(device)
        output = []
        optimizer.zero_grad()
        for clf, label in zip([models[0], models[1]], labels):
            output.append(clf.module.features(x))
        print(type(output))
        final_output = ecls(output[0],output[1])
        print(final_output)
        loss = torch.FloatTensor([0.]).to(device)
        loss += criterion(final_output,y)


import copy
def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    output=[]
    model.train()
    correct, total = 0,0
    
    for (x, y) in tqdm(iterator):
        with autocast(enabled=True):
            x, y = x.to(device), y.to(device)
#             _inp1, _inp2 = x, copy.deepcopy(x)
            optimizer.zero_grad()
            final_output = model(x)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            total += y.size(0)
            
            acc_1, acc_5 = calculate_topk_accuracy(final_output, y)
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = torch.max(final_output.data, 1)
            correct += float(predicted.eq(y.data).cpu().sum())
            total += float(y.size(0))
        
            epoch_loss += loss.item()

    print("Accuracy = {}".format(100. * correct / total))

    epoch_loss /= len(iterator)
    epoch_acc = correct / total * 100
    return epoch_loss, epoch_acc


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for (x, y) in tqdm(iterator):
            x, y = x.to(device), y.to(device)
            final_output = model(x)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            _, predicted = torch.max(final_output.data, 1)
            correct += float(predicted.eq(y.data).cpu().sum())
            total += float(y.size(0))
            epoch_loss += loss.item()

    epoch_loss /= len(iterator)
    epoch_acc = correct / total * 100

    return epoch_loss, epoch_acc


def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    labels = ['model1', 'model2']
    output=[]
    model.train()
    tot = 0
    correct = 0
    for (x, y) in tqdm(iterator):
        with autocast(enabled=True):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            for clf, label in zip([models[0], models[1]], labels):
                output.append(clf.module.features(Variable(x)))
            final_output = ecls(output[0],output[1])
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
#             print(loss)
            tot += y.size(0)
            
            acc_1, acc_5 = calculate_topk_accuracy(final_output, y)
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = final_output.max(1)
            correct += predicted.eq(y).sum().item()
            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
    print("Accuracy = {}".format(100. * correct / tot))

    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)

    return epoch_loss, epoch_acc_1, epoch_acc_5


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0

    model.eval()
    with torch.no_grad():
        for (x, y) in tqdm(iterator):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)

            acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
            epoch_acc_5 += acc_5.item()

    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)

    return epoch_loss, epoch_acc_1, epoch_acc_5